In [1]:
# Calculamos el TMY para un lugar.

import itertools as it

import numpy as np
import pandas as pd

import xarray as xr

import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.interpolate as interp

In [2]:
lat = 19.41
lon = -99.14
path = "../results/NSRDB_2.nc"
ds = xr.open_dataset(path)
df = ds.sel( {"lat": lat, "lon": lon} ).to_dataframe().reset_index(
    ).drop( ["lat", "lon"], axis = 1 ).set_index("time")
#df = df[ ~( (df.index.month == 2) & (df.index.day == 29) ) ]
df = df.drop("DNI", axis = 1)
df

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction
time,,,,,,
2006-01-01 00:00:00,11.8,0.0,68.8,785.0,0.7,198.0
2006-01-01 01:00:00,11.2,0.0,69.9,784.0,0.8,210.0
2006-01-01 02:00:00,10.9,0.0,69.3,784.0,0.7,222.0
2006-01-01 03:00:00,10.6,0.0,68.5,784.0,0.6,234.0
2006-01-01 04:00:00,10.2,0.0,68.1,784.0,0.5,250.0
...,...,...,...,...,...,...
2022-12-31 19:00:00,12.6,0.0,54.8,785.0,0.9,253.0
2022-12-31 20:00:00,11.7,0.0,59.6,786.0,1.0,256.0
2022-12-31 21:00:00,11.5,0.0,60.8,786.0,0.8,263.0


In [3]:
# Modelo NREL-DISC, calcula DNI usando solo GHI.

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

# Huso horario.
TZ = -6

df["Day Angle"] = 6.283185 * ( df.index.dayofyear - 1 ) / 365
df["ETR"] = ( 1370 * ( 1.00011 + 0.034221*np.cos(df["Day Angle"])
    + 0.00128*np.sin(df["Day Angle"]) + 0.000719*np.cos(2*df["Day Angle"])
    +0.000077*np.sin(2*df["Day Angle"]) ) )
df["DEC"] = ( ( 0.006918 - 0.399912 * np.cos(df["Day Angle"])
    + 0.070257*np.sin(df["Day Angle"]) - 0.006758*np.cos(2*df["Day Angle"])
    + 0.000907*np.sin(2*df["Day Angle"]) - 0.002697*np.cos(3*df["Day Angle"])
    + 0.00148*np.sin(3*df["Day Angle"]) ) * 180/np.pi )
df["EQT"] = ( ( 0.000075 + 0.001868*np.cos(df["Day Angle"])
    - 0.032077*np.sin(df["Day Angle"]) - 0.014615*np.cos(2*df["Day Angle"])
    -0.040849*np.sin(2*df["Day Angle"])) * 229.18 )
df["Hour Angle"] = ( 15 * ( df.index.hour - 12
    - 0.5 + df["EQT"]/60 + ((lon-TZ*15)*4)/60 ) )
df["Zenith Angle"] = acos( cos(df["DEC"]) * cos(lat)
    * cos(df["Hour Angle"]) + sin(df["DEC"]) * sin(lat) )
df["AM"] = ( ( 1/cos(df["Zenith Angle"])
    + 0.15/(93.885 - df["Zenith Angle"])**1.253 ) * df["Pressure"]/1013.25 )
df["AM"] = df["AM"].where( df["Zenith Angle"] < 80, 0 )
df["Kt"] = df["GHI"] / ( cos(df["Zenith Angle"]) * df["ETR"] )
df["Kt"] = df["Kt"].where( df["AM"] > 0, 0 )
df["A_1"] = -5.743 + 21.77*df["Kt"] - 27.49*df["Kt"]**2 + 11.56*df["Kt"]**3
df["A_2"] = 0.512 - 1.56*df["Kt"] + 2.286*df["Kt"]**2 - 2.222*df["Kt"]**3
df["A"] = df["A_1"].where( df["Kt"] > 0.6, df["A_2"] ).where( df["Kt"] > 0, 0 )
df["B_1"] = 41.4 - 118.5*df["Kt"] + 66.05*df["Kt"]**2 + 31.9*df["Kt"]**3
df["B_2"] = 0.37 + 0.962*df["Kt"]
df["B"] = df["B_1"].where( df["Kt"] > 0.6, df["B_2"] ).where( df["Kt"] > 0, 0 )
df["C_1"] = -47.01 + 184.2*df["Kt"] - 222*df["Kt"]**2 + 73.81*df["Kt"]**3
df["C_2"] = -0.28 + 0.932*df["Kt"] - 2.048*df["Kt"]**2
df["C"] = df["C_1"].where( df["Kt"] > 0.6, df["C_2"] ).where( df["Kt"] > 0, 0 )
df["D_Kn"] = df["A"] + df["B"] * np.exp( df["C"] * df["AM"] )
df["D_Kn"] = df["D_Kn"].where( df["Kt"] > 0, 0 )
df["Knc"] = ( 0.886 - 0.122*df["AM"] + 0.0121*df["AM"]**2
    - 0.000653*df["AM"]**3 + 0.000014*df["AM"]**4 )
df["Knc"] = df["Knc"].where( df["Kt"] > 0, 0 )
df["DNI"] = df["ETR"] * ( df["Knc"] - df["D_Kn"] )
df["DNI"] = df["DNI"].where( df["Kt"] > 0, 0
    ).where( df["DNI"] > 0, 0 ).round( decimals = 2 )
df = df.drop( [ "Day Angle", "ETR", "DEC", "EQT", "Hour Angle",
    "Zenith Angle", "AM", "Kt", "A_1", "A_2", "A", "B_1",
    "B_2", "B", "C_1", "C_2", "C", "D_Kn", "Knc" ], axis = 1 )

df.head(30)

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI
time,,,,,,,
2006-01-01 00:00:00,11.8,0.0,68.8,785.0,0.7,198.0,0.00
2006-01-01 01:00:00,11.2,0.0,69.9,784.0,0.8,210.0,0.00
2006-01-01 02:00:00,10.9,0.0,69.3,784.0,0.7,222.0,0.00
2006-01-01 03:00:00,10.6,0.0,68.5,784.0,0.6,234.0,0.00
2006-01-01 04:00:00,10.2,0.0,68.1,784.0,0.5,250.0,0.00
2006-01-01 05:00:00,9.9,0.0,67.3,785.0,0.5,266.0,0.00
2006-01-01 06:00:00,9.9,0.0,65.0,785.0,0.4,282.0,0.00
2006-01-01 07:00:00,11.8,28.0,56.3,786.0,0.5,277.0,0.00
2006-01-01 08:00:00,15.7,250.0,44.3,786.0,0.8,261.0,0.00


In [4]:
#df_d = df.resample("D").sum()
#df_d["Error"] = np.abs( 1 - df_d["DNI_DISC"] / df_d["DNI"] )
#print( df_d.groupby(df_d.index.month).mean().describe()["Error"] )
#print( df_d.groupby(df_d.index.year ).mean().describe()["Error"] )
#print( df_d.groupby( [df_d.index.year, df_d.index.month]
#    ).mean().describe()["Error"] )

In [5]:
# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

path = "../results/NSRDB_2.nc"
ds_d = xr.open_dataset(path)

Latitudes = ds_d["lat"].values
Longitudes = ds_d["lon"].values

dfs = []

for lat in Latitudes:
    for lon in Longitudes:
        df_i = ds_d.sel( {"lat": lat, "lon": lon} ).to_dataframe().reset_index(
            ).drop( ["lat", "lon"], axis = 1 ).set_index("time")
        df_i = df_i.drop("DNI", axis = 1)

        # Modelo NREL-DISC, calcula DNI usando solo GHI.

        # Huso horario.
        TZ = -6

        df_i["Day Angle"] = 6.283185 * ( df_i.index.dayofyear - 1 ) / 365
        df_i["ETR"] = ( 1370 * ( 1.00011 + 0.034221*np.cos(df_i["Day Angle"])
            + 0.00128*np.sin(df_i["Day Angle"])
            + 0.000719*np.cos(2*df_i["Day Angle"])
            +0.000077*np.sin(2*df_i["Day Angle"]) ) )
        df_i["DEC"] = ( ( 0.006918 - 0.399912 * np.cos(df_i["Day Angle"])
            + 0.070257*np.sin(df_i["Day Angle"])
            - 0.006758*np.cos(2*df_i["Day Angle"])
            + 0.000907*np.sin(2*df_i["Day Angle"])
            - 0.002697*np.cos(3*df_i["Day Angle"])
            + 0.00148*np.sin(3*df_i["Day Angle"]) ) * 180/np.pi )
        df_i["EQT"] = ( ( 0.000075 + 0.001868*np.cos(df_i["Day Angle"])
            - 0.032077*np.sin(df_i["Day Angle"])
            - 0.014615*np.cos(2*df_i["Day Angle"])
            -0.040849*np.sin(2*df_i["Day Angle"])) * 229.18 )
        df_i["Hour Angle"] = ( 15 * ( df_i.index.hour - 12
            - 0.5 + df_i["EQT"]/60 + ((lon-TZ*15)*4)/60 ) )
        df_i["Zenith Angle"] = acos( cos(df_i["DEC"]) * cos(lat)
            * cos(df_i["Hour Angle"]) + sin(df_i["DEC"]) * sin(lat) )
        df_i["AM"] = ( ( 1/cos(df_i["Zenith Angle"])
            + 0.15/(93.885 - df_i["Zenith Angle"])**1.253 )
            * df_i["Pressure"]/1013.25 )
        df_i["AM"] = df_i["AM"].where( df_i["Zenith Angle"] < 80, 0 )
        df_i["Kt"] = ( df_i["GHI"] / (cos(df_i["Zenith Angle"])*df_i["ETR"]) )
        df_i["Kt"] = df_i["Kt"].where( df_i["AM"] > 0, 0 )
        df_i["A_1"] = ( -5.743 + 21.77*df_i["Kt"]
            - 27.49*df_i["Kt"]**2 + 11.56*df_i["Kt"]**3 )
        df_i["A_2"] = ( 0.512 - 1.56*df_i["Kt"]
            + 2.286*df_i["Kt"]**2 - 2.222*df_i["Kt"]**3 )
        df_i["A"] = ( df_i["A_1"].where( df_i["Kt"] > 0.6, df_i["A_2"]
                ).where( df_i["Kt"] > 0, 0 ) )
        df_i["B_1"] = ( 41.4 - 118.5*df_i["Kt"]
            + 66.05*df_i["Kt"]**2 + 31.9*df_i["Kt"]**3 )
        df_i["B_2"] = 0.37 + 0.962*df_i["Kt"]
        df_i["B"] = df_i["B_1"].where( df_i["Kt"] > 0.6, df_i["B_2"]
            ).where( df_i["Kt"] > 0, 0 )
        df_i["C_1"] = ( -47.01 + 184.2*df_i["Kt"]
            - 222*df_i["Kt"]**2 + 73.81*df_i["Kt"]**3 )
        df_i["C_2"] = -0.28 + 0.932*df_i["Kt"] - 2.048*df_i["Kt"]**2
        df_i["C"] = df_i["C_1"].where( df_i["Kt"] > 0.6, df_i["C_2"]
            ).where( df_i["Kt"] > 0, 0 )
        df_i["D_Kn"] = df_i["A"] + df_i["B"] * np.exp( df_i["C"] * df_i["AM"] )
        df_i["D_Kn"] = df_i["D_Kn"].where( df_i["Kt"] > 0, 0 )
        df_i["Knc"] = ( 0.886 - 0.122*df_i["AM"] + 0.0121*df_i["AM"]**2
            - 0.000653*df_i["AM"]**3 + 0.000014*df_i["AM"]**4 )
        df_i["Knc"] = df_i["Knc"].where( df_i["Kt"] > 0, 0 )
        df_i["DNI"] = df_i["ETR"] * ( df_i["Knc"] - df_i["D_Kn"] )
        df_i["DNI"] = df_i["DNI"].where( df_i["Kt"] > 0, 0
            ).where( df_i["DNI"] > 0, 0 ).round( decimals = 2 )
        df_i = df_i.drop( [ "Day Angle", "ETR", "DEC", "EQT", "Hour Angle",
            "Zenith Angle", "AM", "Kt", "A_1", "A_2", "A", "B_1",
            "B_2", "B", "C_1", "C_2", "C", "D_Kn", "Knc" ], axis = 1 )

        # Convertimos a Dataset.
        df_i = df_i.reset_index()
        df_i["lat" ] = float(lat)
        df_i["lon"] = float(lon)
        dfs.append( df_i.set_index( ["time", "lat", "lon"]
            ).astype( float ).round( decimals = 2 ) )

ds_d.close()

df = pd.concat(dfs)
ds_r = df.to_xarray()
ds_r["lat"] = ds_r["lat"].assign_attrs( standard_name = "latitude",
    long_name = "Latitude", units = "degrees" )
ds_r["lon"] = ds_r["lon"].assign_attrs(standard_name = "longitude",
    long_name = "Longitude", units = "degrees" )
ds_r["time"] = pd.date_range( "01/01/2006 00:00:00",
    "31/12/2022 23:00:00", freq = "H" )
ds_r.to_netcdf("../results/NSRDB_2.nc")

In [6]:
path = "../results/NSRDB.nc"
ds_1 = xr.open_dataset(path)

path = "../results/NSRDB_2.nc"
ds_2 = xr.open_dataset(path)

In [7]:
var = "DNI"
lat = 19.41
lon = -99.14
df_1 = ds_1.sel({"lat": lat, "lon": lon})[var].to_dataframe().reset_index(
    ).drop( ["lat", "lon"], axis = 1 ).set_index("time").resample("D").sum()
df_2 = ds_2.sel({"lat": lat, "lon": lon})[var].to_dataframe().reset_index(
    ).drop( ["lat", "lon"], axis = 1 ).set_index("time").resample("D").sum()
print( df_1.groupby(df_1.index.month).mean().describe() )
print( df_2.groupby(df_2.index.month).mean().describe() )

               DNI
count    12.000000
mean   5902.143388
std    1279.556842
min    3735.168000
25%    4808.297548
50%    6191.273785
75%    6945.931290
max    7491.558074
               DNI
count    12.000000
mean   4247.198451
std     407.478761
min    3416.171392
25%    3974.202884
50%    4373.341279
75%    4606.340743
max    4651.873059


In [8]:
lat = 19.41
lon = -99.18
tilt_A = lat
azimuth_A = 180
path = "../results/NSRDB_2.nc"
ds = xr.open_dataset(path)
df = ds.sel( {"lat": lat, "lon": lon} ).to_dataframe().reset_index(
    ).drop( ["lat", "lon"], axis = 1 ).set_index("time")
df = df[ ~( (df.index.month == 2) & (df.index.day == 29) ) ]
df = df.shift( periods = 1, freq = "H")
df.head(30)

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI
time,,,,,,,
2006-01-01 01:00:00,11.7,0.0,68.4,784.0,0.7,198.0,0.00
2006-01-01 02:00:00,11.2,0.0,68.9,783.0,0.8,210.0,0.00
2006-01-01 03:00:00,10.9,0.0,68.2,783.0,0.7,222.0,0.00
2006-01-01 04:00:00,10.6,0.0,67.3,783.0,0.6,234.0,0.00
2006-01-01 05:00:00,10.2,0.0,66.8,783.0,0.5,250.0,0.00
2006-01-01 06:00:00,9.9,0.0,65.8,784.0,0.5,266.0,0.00
2006-01-01 07:00:00,9.9,0.0,63.4,784.0,0.4,282.0,0.00
2006-01-01 08:00:00,11.8,27.0,54.8,785.0,0.4,277.0,0.00
2006-01-01 09:00:00,15.6,250.0,43.7,785.0,0.7,261.0,0.00


In [9]:
# Modelo de Pérez de Cielo Difuso.

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

# Huso horario.
TZ = -6

df["Day Angle"] = 6.283185 * ( df.index.dayofyear - 1 ) / 365
df["ETR"] = ( 1370 * ( 1.00011 + 0.034221*np.cos(df["Day Angle"])
    + 0.00128*np.sin(df["Day Angle"]) + 0.000719*np.cos(2*df["Day Angle"])
    +0.000077*np.sin(2*df["Day Angle"]) ) )
df["DEC"] = ( ( 0.006918 - 0.399912 * np.cos(df["Day Angle"])
    + 0.070257*np.sin(df["Day Angle"]) - 0.006758*np.cos(2*df["Day Angle"])
    + 0.000907*np.sin(2*df["Day Angle"]) - 0.002697*np.cos(3*df["Day Angle"])
    + 0.00148*np.sin(3*df["Day Angle"]) ) * 180/np.pi )
df["EQT"] = ( ( 0.000075 + 0.001868*np.cos(df["Day Angle"])
    - 0.032077*np.sin(df["Day Angle"]) - 0.014615*np.cos(2*df["Day Angle"])
    -0.040849*np.sin(2*df["Day Angle"])) * 229.18 )
df["Hour Angle"] = ( 15 * ( df.index.hour - 12
    - 0.5 + df["EQT"]/60 + ((lon-TZ*15)*4)/60 ) )
df["Zenith Angle"] = acos( cos(df["DEC"])*cos(lat)*cos(df["Hour Angle"])
    + sin(df["DEC"])*sin(lat) )
df["Azimuth Angle"] = acos( (sin(df["DEC"])-cos(df["Zenith Angle"])*sin(lat))
    / (sin(df["Zenith Angle"])*cos(lat)) )
df["Azimuth Angle"] = df["Azimuth Angle"].where(
    df["Hour Angle"] < 0, 360 - df["Azimuth Angle"] )
df["AOI"] = acos( cos(df["Zenith Angle"])*cos(tilt_A)
    + sin(df["Zenith Angle"])*sin(tilt_A)*cos(df["Azimuth Angle"]-azimuth_A) )
df["AM"] = ( ( 1/cos(df["Zenith Angle"])
    + 0.15/(93.885 - df["Zenith Angle"])**1.253 )
    * df["Pressure"]/1013.25 )
df["AM"] = df["AM"].where( df["Zenith Angle"] < 80, 0 )
df["DHI"] = df["GHI"] - df["DNI"] * cos(df["Zenith Angle"])
K = 5.535e-6
def bins(x):
    if   x < 1.065: return 1
    elif x < 1.230: return 2
    elif x < 1.500: return 3
    elif x < 1.950: return 4
    elif x < 2.800: return 5
    elif x < 4.500: return 6
    elif x < 6.200: return 7
    else          : return 8
df["epsilon"] = ( ( (df["DHI"]+df["DNI"])/df["DHI"] + K*df["Zenith Angle"]**3 )
    / ( 1 + K*df["Zenith Angle"]**3 ) ).apply(bins)
Perez = pd.read_csv("Perez.csv", index_col = "bin" )
def get_Perez(epsilon): return Perez.loc[epsilon]
Ea = 1367
df["Delta"] = df["DHI"] * df["AM"] / Ea
df[ Perez.columns ] = df["epsilon"].apply(get_Perez)
df["F1"] = ( df["f11"] + df["f12"]*df["Delta"]
    + np.radians(df["Zenith Angle"])*df["f13"] )
df["F1"] = df["F1"].where( df["Zenith Angle"] > 0, 0 )
df["F2"] = ( df["f21"] + df["f22"]*df["Delta"]
    + np.radians(df["Zenith Angle"])*df["f23"] )
df["a"] = cos(df["AOI"])
df["a"] = df["a"].where( df["a"] > 0, 0 )
df["b"] = cos(df["AOI"])
df["b"] = df["b"].where( df["b"] > 0, cos(85) )
df["I_d"] = ( ( df["DHI"] * ( (1-df["F1"]) * ((1+cos(tilt_A))/2)
    + df["F1"]*df["a"]/df["b"] + df["F2"]*sin(tilt_A)
    ) ).round( decimals = 2 ) )
df["I_b"] = (df["DNI"]*cos(df["AOI"])).round( decimals = 2 )
df["POA"] = df["I_b"] + df["I_d"]
df = df.drop( ["Day Angle", "ETR", "DEC", "EQT", "Hour Angle",
    #"Zenith Angle", "Azimuth Angle",
    "AOI", "AM", "DHI", "epsilon", "Delta",
    "F1", "F2", "a", "b"] + list(Perez.columns), axis = 1 )

df.head(25)

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI,Zenith Angle,Azimuth Angle,I_d,I_b,POA
time,,,,,,,,,,,,
2006-01-01 01:00:00,11.7,0.0,68.4,784.0,0.7,198.0,0.00,175.717526,148.850278,0.00,-0.00,0.00
2006-01-01 02:00:00,11.2,0.0,68.9,783.0,0.8,210.0,0.00,167.712858,109.487650,0.00,-0.00,0.00
2006-01-01 03:00:00,10.9,0.0,68.2,783.0,0.7,222.0,0.00,154.061202,103.003042,0.00,-0.00,0.00
2006-01-01 04:00:00,10.6,0.0,67.3,783.0,0.6,234.0,0.00,140.265734,103.047428,0.00,-0.00,0.00
2006-01-01 05:00:00,10.2,0.0,66.8,783.0,0.5,250.0,0.00,126.529468,104.820576,0.00,-0.00,0.00
2006-01-01 06:00:00,9.9,0.0,65.8,784.0,0.5,266.0,0.00,112.939624,107.566382,0.00,-0.00,0.00
2006-01-01 07:00:00,9.9,0.0,63.4,784.0,0.4,282.0,0.00,99.589876,111.200264,0.00,-0.00,0.00
2006-01-01 08:00:00,11.8,27.0,54.8,785.0,0.4,277.0,0.00,86.614450,115.902788,25.32,0.00,25.32
2006-01-01 09:00:00,15.6,250.0,43.7,785.0,0.7,261.0,0.00,74.228065,122.075785,240.22,0.00,240.22


In [10]:
# NOCT Cell Temperature Model.

T_NOCT    = 44 # °C
I_mp      = 14.99 # A
V_mp      = 35.56  # V
A_m       = 1.096*2.384 # m^2
eff_ref   = I_mp * V_mp / (1000 * A_m)
tau_alpha = 0.9

# Ajuste de viento.
#v = 0.61 # Dos pisos.
v = 0.51 # Un piso.

# Ajuste de montaje.
T_adj = 2   + T_NOCT # Building integrated,
# greater than 3.5 in, or groud/rack mounted
#T_adj = 2  + T_NOCT # 2.5 to 3.5 in
#T_adj = 6  + T_NOCT # 1.5 to 2.5 in
#T_adj = 11 + T_NOCT # 0.5 to 1.5 in
#T_adj = 18 + T_NOCT # less than 0.5 in

df["Cell Temperature"] = ( df["Temperature"] + df["POA"] / 800 * (T_adj-20)
    * (1-eff_ref/tau_alpha) * ( 9.5 / (5.7+3.8*v*df["Wind Speed"]) )
    ).round( decimals = 2 )

df.head(25)

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI,Zenith Angle,Azimuth Angle,I_d,I_b,POA,Cell Temperature
time,,,,,,,,,,,,,
2006-01-01 01:00:00,11.7,0.0,68.4,784.0,0.7,198.0,0.00,175.717526,148.850278,0.00,-0.00,0.00,11.70
2006-01-01 02:00:00,11.2,0.0,68.9,783.0,0.8,210.0,0.00,167.712858,109.487650,0.00,-0.00,0.00,11.20
2006-01-01 03:00:00,10.9,0.0,68.2,783.0,0.7,222.0,0.00,154.061202,103.003042,0.00,-0.00,0.00,10.90
2006-01-01 04:00:00,10.6,0.0,67.3,783.0,0.6,234.0,0.00,140.265734,103.047428,0.00,-0.00,0.00,10.60
2006-01-01 05:00:00,10.2,0.0,66.8,783.0,0.5,250.0,0.00,126.529468,104.820576,0.00,-0.00,0.00,10.20
2006-01-01 06:00:00,9.9,0.0,65.8,784.0,0.5,266.0,0.00,112.939624,107.566382,0.00,-0.00,0.00,9.90
2006-01-01 07:00:00,9.9,0.0,63.4,784.0,0.4,282.0,0.00,99.589876,111.200264,0.00,-0.00,0.00,9.90
2006-01-01 08:00:00,11.8,27.0,54.8,785.0,0.4,277.0,0.00,86.614450,115.902788,25.32,0.00,25.32,12.73
2006-01-01 09:00:00,15.6,250.0,43.7,785.0,0.7,261.0,0.00,74.228065,122.075785,240.22,0.00,240.22,23.73


In [11]:
# Simple efficiency module model.

F_d = 1
eff_T = -0.34

df["I_r"] = 0

df["G"] = df["I_b"] + F_d * ( df["I_d"] + df["I_r"] )
df["P_mp"] = ( df["G"]*eff_ref*A_m * (1+eff_T/100*(df["Cell Temperature"]-25))
    ).round( decimals = 2 )
df = df.drop(["I_b", "I_d", "I_r", "G"], axis = 1)

df.head(25)

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI,Zenith Angle,Azimuth Angle,POA,Cell Temperature,P_mp
time,,,,,,,,,,,,
2006-01-01 01:00:00,11.7,0.0,68.4,784.0,0.7,198.0,0.00,175.717526,148.850278,0.00,11.70,0.00
2006-01-01 02:00:00,11.2,0.0,68.9,783.0,0.8,210.0,0.00,167.712858,109.487650,0.00,11.20,0.00
2006-01-01 03:00:00,10.9,0.0,68.2,783.0,0.7,222.0,0.00,154.061202,103.003042,0.00,10.90,0.00
2006-01-01 04:00:00,10.6,0.0,67.3,783.0,0.6,234.0,0.00,140.265734,103.047428,0.00,10.60,0.00
2006-01-01 05:00:00,10.2,0.0,66.8,783.0,0.5,250.0,0.00,126.529468,104.820576,0.00,10.20,0.00
2006-01-01 06:00:00,9.9,0.0,65.8,784.0,0.5,266.0,0.00,112.939624,107.566382,0.00,9.90,0.00
2006-01-01 07:00:00,9.9,0.0,63.4,784.0,0.4,282.0,0.00,99.589876,111.200264,0.00,9.90,0.00
2006-01-01 08:00:00,11.8,27.0,54.8,785.0,0.4,277.0,0.00,86.614450,115.902788,25.32,12.73,14.06
2006-01-01 09:00:00,15.6,250.0,43.7,785.0,0.7,261.0,0.00,74.228065,122.075785,240.22,23.73,128.60
